In [67]:
import pandas as pd
import glob
import dropbox as db
import matplotlib.pyplot as plt
import seaborn as sns

In [74]:
!ln -s /Users/matthiashugli/Virtualenvs/youth-base/youth-base/config.py config.py
from config import dbx_token
dbx = db.Dropbox(dbx_token)

event_path = '/Users/matthiashugli/Dropbox/bucket/dataset-wyscout/events_Germany.json'


events = pd.DataFrame()
for filename in glob.glob(event_path):
    file = pd.read_json(filename)
    file.insert(1, 'filename', filename)
    events = events.append(file)



ln: config.py: File exists


In [75]:
player_path = '/Users/matthiashugli/Dropbox/bucket/dataset-wyscout/players.json'

players = pd.DataFrame()
player_file = pd.read_json(player_path)
players = players.append(player_file)

In [76]:
players['position'] = players.role.apply(pd.Series)['name']
players = players[players['position'] == 'Goalkeeper']
players.drop(columns=['passportArea', 'weight', 'middleName', 'currentTeamId', 'role', 'birthArea', 'shortName', 'currentNationalTeamId'], inplace=True)

In [77]:
events = events.merge(players, left_on=events.playerId, right_on=players.wyId, how='inner')

In [78]:
events = events.explode('tags')
#events['tag_id'] = [value.get('id') for key, value in events['tags'].items()]
events.head()

,key_0,eventId,filename,subEventName,tags,playerId,positions,matchId,eventName,teamId,...,eventSec,subEventId,id,firstName,lastName,birthDate,height,wyId,foot,position
0,14768,8,/Users/matthiashugli/Dropbox/bucket/dataset-wy...,Simple pass,{'id': 1801},14768,"[{'y': 46, 'x': 6}, {'y': 10, 'x': 20}]",2516739,Pass,2446,...,10.786491,85,179896445,Bernd,Leno,1992-03-04,190,14768,right,Goalkeeper
1,14768,3,/Users/matthiashugli/Dropbox/bucket/dataset-wy...,Goal kick,NaN,14768,"[{'y': 100, 'x': 100}, {'y': 42, 'x': 16}]",2516739,Free Kick,2446,...,181.205506,34,179896508,Bernd,Leno,1992-03-04,190,14768,right,Goalkeeper
2,14768,8,/Users/matthiashugli/Dropbox/bucket/dataset-wy...,Simple pass,{'id': 1801},14768,"[{'y': 42, 'x': 6}, {'y': 10, 'x': 3}]",2516739,Pass,2446,...,186.589156,85,179896511,Bernd,Leno,1992-03-04,190,14768,right,Goalkeeper
3,14768,8,/Users/matthiashugli/Dropbox/bucket/dataset-wy...,Launch,{'id': 1801},14768,"[{'y': 34, 'x': 5}, {'y': 28, 'x': 45}]",2516739,Pass,2446,...,189.818782,84,179896514,Bernd,Leno,1992-03-04,190,14768,right,Goalkeeper
4,14768,8,/Users/matthiashugli/Dropbox/bucket/dataset-wy...,High pass,{'id': 1801},14768,"[{'y': 26, 'x': 9}, {'y': 87, 'x': 39}]",2516739,Pass,2446,...,236.008683,83,179896540,Bernd,Leno,1992-03-04,190,14768,right,Goalkeeper


In [55]:
shot_against = events[events['eventId'] == 9]
shot_against.groupby(['eventId']).tags.apply(lambda x: pd.DataFrame(x.values[0])).reset_index()

,eventId,level_1,id
0,9,0,1206
1,9,1,1801


In [59]:
reduced = shot_against.head()
#print(reduced.head())
exploded = reduced.explode('tags')
#exploded.head()
exploded['tag_id'] = [value.get('id') for key, value in exploded['tags'].items()]
exploded.reset_index(inplace=True)
exploded.head()

,index,key_0,eventId,filename,subEventName,tags,playerId,positions,matchId,eventName,...,subEventId,id,firstName,lastName,birthDate,height,wyId,foot,position,tag_id
0,5,14768,9,/Users/matthiashugli/Dropbox/bucket/dataset-wy...,Save attempt,{'id': 1206},14768,"[{'y': 100, 'x': 100}, {'y': 34, 'x': 17}]",2516739,Save attempt,...,91,179896543,Bernd,Leno,1992-03-04,190,14768,right,Goalkeeper,1206
1,5,14768,9,/Users/matthiashugli/Dropbox/bucket/dataset-wy...,Save attempt,{'id': 1801},14768,"[{'y': 100, 'x': 100}, {'y': 34, 'x': 17}]",2516739,Save attempt,...,91,179896543,Bernd,Leno,1992-03-04,190,14768,right,Goalkeeper,1801
2,6,14768,9,/Users/matthiashugli/Dropbox/bucket/dataset-wy...,Reflexes,{'id': 101},14768,"[{'y': 100, 'x': 100}, {'y': 41, 'x': 5}]",2516739,Save attempt,...,90,179896637,Bernd,Leno,1992-03-04,190,14768,right,Goalkeeper,101
3,6,14768,9,/Users/matthiashugli/Dropbox/bucket/dataset-wy...,Reflexes,{'id': 1205},14768,"[{'y': 100, 'x': 100}, {'y': 41, 'x': 5}]",2516739,Save attempt,...,90,179896637,Bernd,Leno,1992-03-04,190,14768,right,Goalkeeper,1205
4,6,14768,9,/Users/matthiashugli/Dropbox/bucket/dataset-wy...,Reflexes,{'id': 1802},14768,"[{'y': 100, 'x': 100}, {'y': 41, 'x': 5}]",2516739,Save attempt,...,90,179896637,Bernd,Leno,1992-03-04,190,14768,right,Goalkeeper,1802


In [6]:
#events.head()
#players.head()
#events.count()
#events.groupby(['eventName', 'subEventName']).eventId.count()
events[events.eventName == 'Save attempt'].tail()

,key_0,eventId,filename,subEventName,tags,playerId,positions,matchId,eventName,teamId,...,eventSec,subEventId,id,firstName,lastName,birthDate,height,wyId,foot,position
26675,14822,9,/Users/matthiashugli/Dropbox/bucket/dataset-wy...,Reflexes,"[{'id': 101}, {'id': 1205}, {'id': 1802}]",14822,"[{'y': 100, 'x': 100}, {'y': 40, 'x': 10}]",2517020,Save attempt,2481,...,2517.330781,90,247152490,Andreas,Luthe,1987-03-10,195,14822,right,Goalkeeper
26698,14822,9,/Users/matthiashugli/Dropbox/bucket/dataset-wy...,Reflexes,"[{'id': 1203}, {'id': 1801}]",14822,"[{'y': 100, 'x': 100}, {'y': 42, 'x': 4}]",2517039,Save attempt,2481,...,2294.344382,90,251254015,Andreas,Luthe,1987-03-10,195,14822,right,Goalkeeper
26704,14822,9,/Users/matthiashugli/Dropbox/bucket/dataset-wy...,Reflexes,"[{'id': 101}, {'id': 1204}, {'id': 1802}]",14822,"[{'y': 100, 'x': 100}, {'y': 53, 'x': 10}]",2517039,Save attempt,2481,...,244.789908,90,251254292,Andreas,Luthe,1987-03-10,195,14822,right,Goalkeeper
26717,14822,9,/Users/matthiashugli/Dropbox/bucket/dataset-wy...,Reflexes,"[{'id': 101}, {'id': 1201}, {'id': 1802}]",14822,"[{'y': 100, 'x': 100}, {'y': 67, 'x': 10}]",2517039,Save attempt,2481,...,1203.607650,90,251254535,Andreas,Luthe,1987-03-10,195,14822,right,Goalkeeper
26745,333477,9,/Users/matthiashugli/Dropbox/bucket/dataset-wy...,Reflexes,"[{'id': 1205}, {'id': 1801}]",333477,"[{'y': 0, 'x': 0}, {'y': 35, 'x': 18}]",2517040,Save attempt,2449,...,1860.826622,90,251299375,Alexander,N\u00fcbel,1996-09-30,193,333477,right,Goalkeeper
